In [ ]:
# imports
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, silhouette_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.cluster import KMeans


# Task 1

In [ ]:
# check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# unzip folder
!unzip -q "Dataset 1.zip" -d "/content"

unzip:  cannot find or open Dataset 1.zip, Dataset 1.zip.zip or Dataset 1.zip.ZIP.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Define data transforms for augmentation and normalization
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset using ImageFolder
data_dir = '/content/Dataset 1/Colorectal Cancer '
medical_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Split dataset into 70% training and 30% testing sets
train_size = int(0.6 * len(medical_dataset))
val_size = int(0.2 * len(medical_dataset))
test_size = len(medical_dataset) - train_size - val_size
train_set, val_set, test_set = random_split(medical_dataset, [train_size, val_size, test_size])

# Create DataLoaders for training, validation, and test sets
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)

class_names = medical_dataset.classes  # ['MUS', 'NORM', 'STR']

In [ ]:
# Load a ResNet18 model
model = models.resnet18(pretrained=False)

# Replace the final layer to match the number of classes (3: MUS, NORM, STR)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # 3 output classes

# Move model to GPU if available
model = model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs):
    best_model_wts = None
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        # Validation phase
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100 * correct / total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = model.state_dict()

        print(f"Epoch [{epoch + 1}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}% "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%")

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_losses, train_accuracies, val_losses, val_accuracies

In [ ]:
# train the model
_, loss_list, accuracy_list, _, _ = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10)

In [ ]:
# plot the loss and accuracy over epochs
epochs = range(1, len(loss_list) + 1)

plt.figure(figsize=(10,5))

# plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss_list, '-o', label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs. Epochs')
plt.legend()

# plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy_list, '-o', label='Accuracy', color='green')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Epochs')
plt.legend()

# display the plots
plt.tight_layout()
plt.show()

In [ ]:
target_names = ['MUS', 'NORM', 'STR']

def evaluate_model(model, test_loader):
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)


            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)


            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())


    accuracy = accuracy_score(all_labels, all_preds)


    if target_names is not None:
        print("Classification Report:")
        print(classification_report(all_labels, all_preds, target_names=target_names, zero_division=0))
    else:
        print("Classification Report:")
        print(classification_report(all_labels, all_preds, zero_division=0))

    print(f"Accuracy: {accuracy:.2f}")

    return accuracy

In [ ]:
#evaluate the model
evaluate_model(model, test_loader)

In [ ]:
def extract_features(model, dataloader):
    model.eval()
    features = []
    labels = []

    with torch.no_grad():
        for inputs, label in dataloader:
            inputs = inputs.to(device)
            output = model(inputs)  # Extract features from the final FC layer
            features.append(output.cpu().numpy())
            labels.extend(label.numpy())

    return np.concatenate(features), np.array(labels)

# Extract features from the test set
features, labels = extract_features(model, test_loader)

# Apply t-SNE for dimensionality reduction
tsne = TSNE(n_components=2, random_state=42)
features_2d = tsne.fit_transform(features)

# Get the Silhouette score
kmeans = KMeans(n_clusters=len(np.unique(labels)), random_state=42)
kmeans_labels = kmeans.fit_predict(features)

sil_score = silhouette_score(features, kmeans_labels)
print(f"Silhouette Score: {sil_score}")

# Plot the t-SNE result
def plot_tsne(features, labels, class_names, title="t-SNE of CNN Features"):
    plt.figure(figsize=(10, 8))
    for class_id, class_name in enumerate(class_names):
        idx = labels == class_id
        plt.scatter(features[idx, 0], features[idx, 1], label=class_name, alpha=0.7)
    plt.legend()
    plt.title(title)
    plt.show()

plot_tsne(features_2d, labels, class_names)


In [ ]:
# Feature Map
class FeatureExtractor:
    def __init__(self, model, layer_name):
        self.model = model
        self.layer_name = layer_name
        self.feature_map = None
        self.hook = self._register_hook()

    def _register_hook(self):
        layer = dict(self.model.named_modules())[self.layer_name]
        hook = layer.register_forward_hook(self._hook_fn)
        return hook

    def _hook_fn(self, module, input, output):
        self.feature_map = output.detach()

    def remove_hook(self):
        self.hook.remove()

    def get_feature_map(self):
        return self.feature_map


# Grad-CAM Class
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.feature_maps = None

        self.feature_extractor = FeatureExtractor(model, target_layer)
        self.target_layer = dict(model.named_modules())[target_layer]
        self.target_layer.register_backward_hook(self._save_gradients)

    def _save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def forward(self, input_image):
        return self.model(input_image)

    def generate_cam(self, input_image, class_idx=None):
        # Forward pass
        self.model.zero_grad()
        output = self.forward(input_image)
        if class_idx is None:
            class_idx = torch.argmax(output)

        # Backward pass
        self.model.zero_grad()
        output[:, class_idx].backward()

        # Get the feature maps and gradients
        feature_maps = self.feature_extractor.get_feature_map()
        gradients = self.gradients

        # Compute the weights for each feature map
        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
        cam = torch.sum(weights * feature_maps, dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = cam.squeeze().cpu().detach().numpy()

        # Normalize the CAM to [0, 1]
        cam = cv2.resize(cam, (input_image.shape[3], input_image.shape[2]))
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))

        return cam


# Overlay CAM on Image and Save
def overlay_cam_on_image(original_image, cam, save_path, alpha=0.5):
    # Convert CAM to BGR and resize
    cam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    cam = cv2.cvtColor(cam, cv2.COLOR_BGR2RGB)

    # Convert original image to numpy and normalize it to [0, 255]
    original_image = original_image.squeeze().permute(1, 2, 0).cpu().numpy()
    original_image = (original_image * 255).astype(np.uint8)  # Convert to uint8

    # Overlay the CAM on the image
    overlayed_image = cv2.addWeighted(original_image, alpha, cam, 1 - alpha, 0)

    # Save the overlayed image
    plt.imsave(save_path, overlayed_image)
    print(f"Saved Grad-CAM image to: {save_path}")



# Visualize and Save Grad-CAMs for All Batches
def visualize_and_save_gradcam_all_batches(model, test_loader, target_layer='layer4', device='cuda', save_dir='gradcam_outputs'):
    model.eval() #Set the model to evaluation mode
    os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

    grad_cam = GradCAM(model, target_layer)

    for batch_idx, (inputs, labels) in enumerate(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Loop over each image in the batch
        for i in range(inputs.size(0)):
            input_image = inputs[i].unsqueeze(0)
            cam = grad_cam.generate_cam(input_image)

            # Define the save path for each image
            save_path = os.path.join(save_dir, f"batch_{batch_idx}_image_{i}_true_{labels[i].item()}_pred_{torch.argmax(model(inputs[i].unsqueeze(0))).item()}.png")

            # Save CAM overlayed image
            overlay_cam_on_image(inputs[i], cam, save_path)

            # Optionally print the label and predicted class for logging
            print(f"Batch {batch_idx} - True label: {labels[i].item()} - Predicted label: {torch.argmax(model(inputs[i].unsqueeze(0))).item()}")


# Example Usage
# Assuming `model` and `test_loader` are already defined and `device` is either 'cuda' or 'cpu'.
visualize_and_save_gradcam_all_batches(model, test_loader, target_layer='layer4', device='cuda', save_dir='gradcam_outputs')


## Evaluating different hyperparameters

In [ ]:
# Evaluate hyperparameters of the model
from itertools import product

# Hyperparameter tuning
learning_rates = [0.1, 0.01, 0.001, 0.0001]
batch_sizes = [8, 16, 32, 64]
num_epochs = 10
best_params = {}
best_val_acc = 0.0

for lr, batch_size in product(learning_rates, batch_sizes):
    print(f"Testing Hyperparameters: LR={lr}, Batch Size={batch_size}")
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4)

    # Initialize model, loss, and optimizer
    model = models.resnet18(pretrained=False)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, len(medical_dataset.classes))  # Adjust output layer for 3 classes
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train and evaluate model
    model, _, _, _, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs)
    max_val_acc = max(val_accuracies)

    if max_val_acc > best_val_acc:
        best_val_acc = max_val_acc
        best_params = {'learning_rate': lr, 'batch_size': batch_size}

print(f"Best Hyperparameters: {best_params}, Best Validation Accuracy: {best_val_acc:.2f}%")


# Task 2

## Use model trained on Colorectal cancer

In [ ]:
# unzip folder
!unzip -q "Dataset 2.zip" -d "/content"

In [ ]:
# unzip folder
!unzip -q "Dataset 3.zip" -d "/content"

In [ ]:
# Remove the classification head from the Task 1 model
encoder_task1 = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the final layer
encoder_task1.eval()  # Set to evaluation mode

In [ ]:
# Save the encoder
torch.save(encoder_task1, 'task1_encoder.pth')

# Reload if needed
encoder_task1 = torch.load('task1_encoder.pth')
encoder_task1.eval()

In [ ]:
# Define transforms (same as Task 1 for consistency)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
data_dir2 = '/content/Dataset 2/Prostate Cancer'
data_dir3 = '/content/Dataset 3/Animal Faces'

dataset2 = datasets.ImageFolder(root=data_dir2, transform=data_transforms)
dataset3 = datasets.ImageFolder(root=data_dir3, transform=data_transforms)

dataloader2 = DataLoader(dataset2, batch_size=32, shuffle=False)
dataloader3 = DataLoader(dataset3, batch_size=32, shuffle=False)


In [ ]:
# Extract features for Dataset 2
features2_task1, labels2_task1 = extract_features(encoder_task1, dataloader2)

# Extract features for Dataset 3
features3_task1, labels3_task1 = extract_features(encoder_task1, dataloader3)


In [ ]:
# Load ImageNet-pretrained encoder (e.g., ResNet-18)
imagenet_encoder = models.resnet18(pretrained=True)
imagenet_encoder = torch.nn.Sequential(*list(imagenet_encoder.children())[:-1])
imagenet_encoder.eval()

# Extract features for Dataset 2
features2_imagenet, labels2_imagenet = extract_features(imagenet_encoder, dataloader2)

# Extract features for Dataset 3
features3_imagenet, labels3_imagenet = extract_features(imagenet_encoder, dataloader3)


In [ ]:
# Task 1 Encoder on Dataset 2
features2_task1_tsne = np.squeeze(tsne.fit_transform(np.squeeze(features2_task1), labels2_task1))
plot_tsne(features2_task1_tsne, labels2_task1, dataset2.classes)

# Task 1 Encoder on Dataset 3
features3_task1_tsne = tsne.fit_transform(np.squeeze(features3_task1), labels3_task1)
plot_tsne(features3_task1_tsne, labels3_task1, dataset3.classes)

# ImageNet Encoder on Dataset 2
features2_imagenet_tsne = tsne.fit_transform(np.squeeze(features2_imagenet), labels2_imagenet)
plot_tsne(features2_imagenet_tsne, labels2_imagenet, dataset2.classes)

# ImageNet Encoder on Dataset 3
features3_imagenet_tsne = tsne.fit_transform(np.squeeze(features3_imagenet), labels3_imagenet)
plot_tsne(features3_imagenet_tsne, labels3_imagenet, dataset3.classes)


<h1>Classification of Dataset Using a Machine Learning Technique (SVM)</h1>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Dataset 2: Using features extracted by Task 1 encoder
X2 = features2_task1_2d
y2 = labels2_task1_2d


# Split data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X2_train, y2_train)

# Make predictions
y2_pred = svm_clf.predict(X2_test)

X2_tsne = tsne.fit_transform(X2_test, y2_test)
plot_tsne(X2_tsne, y2_pred, dataset2.classes)


# Evaluate the performance
print("Dataset 2 (Task 1 Encoder) - SVM Results")
print("Accuracy:", accuracy_score(y2_test, y2_pred))
print("Classification Report:\n", classification_report(y2_test, y2_pred))
# Dataset 3: Using features extracted by Task 1 encoder
X3 = np.squeeze(features3_task1)
y3 = np.squeeze(labels3_task1)
# Split data into training and testing sets
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X3_train, y3_train)

# Make predictions
y3_pred = svm_clf.predict(X3_test)

X3_tsne = tsne.fit_transform(X3_test, y3_test)
plot_tsne(X3_tsne, y3_pred, dataset2.classes)

# Evaluate the performance
print("Dataset 3 (Task 1 Encoder) - SVM Results")
print("Accuracy:", accuracy_score(y3_test, y3_pred))
print("Classification Report:\n", classification_report(y3_test, y3_pred))

In [ ]:
# Dataset 2: Using features extracted by ImageNet encoder
X2 = np.squeeze(features2_imagenet)
y2 = np.squeeze(labels2_imagenet)

# Split data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X2_train, y2_train)

# Make predictions
y2_pred = svm_clf.predict(X2_test)

X2_tsne = tsne.fit_transform(X2_test, y2_test)
plot_tsne(X2_tsne, y2_pred, dataset2.classes)

# Evaluate the performance
print("Dataset 2 (ImageNet Encoder) - SVM Results")
print("Accuracy:", accuracy_score(y2_test, y2_pred))
print("Classification Report:\n", classification_report(y2_test, y2_pred))

# Dataset 3: Using features extracted by ImageNet encoder
X3 = np.squeeze(features3_imagenet)
y3 = np.squeeze(labels3_imagenet)

# Split data into training and testing sets
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X3_train, y3_train)

# Make predictions
y3_pred = svm_clf.predict(X3_test)

X3_tsne = tsne.fit_transform(X3_test, y3_test)
plot_tsne(X3_tsne, y3_pred, dataset2.classes)

# Evaluate the performance
print("Dataset 3 (ImageNet Encoder) - SVM Results")
print("Accuracy:", accuracy_score(y3_test, y3_pred))
print("Classification Report:\n", classification_report(y3_test, y3_pred))